In [34]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [35]:
import numpy as np
np.bool = np.bool_

In [36]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [37]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [38]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [39]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executorEnv.PYTHONPATH", "/path/to/your/venv/lib/python3.11/site-packages") \

    .enableHiveSupport()
    .getOrCreate()
)

In [40]:
trump_tweet_with_sentiment_analysis = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/gold/trump_btc/trump_tweet_with_sentiment_analysis").cache()
btc_price = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/").cache()
btc_price = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/").cache()


In [41]:
btc_price.orderBy("date").show(5)

+------------+----+----+----+-----+------+-------------------+
|   Timestamp|Open|High| Low|Close|Volume|               date|
+------------+----+----+----+-----+------+-------------------+
|1.32541206E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:01:00|
|1.32541212E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:02:00|
|1.32541218E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:03:00|
|1.32541224E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:04:00|
| 1.3254123E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:05:00|
+------------+----+----+----+-----+------+-------------------+
only showing top 5 rows



In [42]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|2011-08-22 22:33:46|    193.0|1.057394600749383...|    false|   299.0|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|    105.0|1.111727130155458...|    false|   167.0|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|     23.0|1.122313901285130...|    false|    80.0|@BarackObama---th...|          -0.25|
|2011-09-20 15:17:36|     11.0|1.161389434178109...|    false|    38.0|"@BarackObama's m...|            0.0|
|2011-10-07 21:21:0

In [44]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|2011-08-22 22:33:46|    193.0|1.057394600749383...|    false|   299.0|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|    105.0|1.111727130155458...|    false|   167.0|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|     23.0|1.122313901285130...|    false|    80.0|@BarackObama---th...|          -0.25|
|2011-09-20 15:17:36|     11.0|1.161389434178109...|    false|    38.0|"@BarackObama's m...|            0.0|
|2011-10-07 21:21:0

In [45]:
from pyspark.sql.functions import date_format

#removing the seconds from both DF

trump_tweet_with_sentiment_analysis = trump_tweet_with_sentiment_analysis.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))
btc_price = btc_price.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))


In [46]:
trump_tweet_with_sentiment_analysis.show(5)
btc_price.show(5)

+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|            date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|2024-10-31 17:16|   8601.0|1.134029284408450...|    false|  2282.0|"I would like to ...|         0.8652|
|2024-09-30 17:11|   8655.0|1.132271418381557...|    false|  2516.0|"I promised to Ma...|         0.9151|
|2024-09-24 17:02|      0.0|1.131931321423966...|     true|     0.0|"RT @realtrumpcoi...|            0.0|
|2024-09-22 00:22|   7797.0| 1.13177878498912E17|    false|  2029.0|"https://justthen...|            0.0|
|2024-09-21 23:31|   7113.0|1.131776764644532...|    false|  1909.0|"“Trump Credits S...|          0.886|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
only showing top 5 rows

+------------+-------

In [47]:
from pyspark.sql.functions import col

#filtering the tweets with sentiment +- 0.5
filtered_tweets = trump_tweet_with_sentiment_analysis.filter((trump_tweet_with_sentiment_analysis.sentiment_score > 0.5) | (trump_tweet_with_sentiment_analysis.sentiment_score < -0.5))
#filtering because we only got data from 2012
filtered_tweets = filtered_tweets.filter(col("date") >= "2012-01-01")

filtered_tweets.orderBy('date').show()

+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|            date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|2018-12-17 14:27|  92663.0|1.074657278974939...|    false| 20421.0|It is incredible ...|         0.6893|
|2019-04-30 19:56|  47900.0|1.123285120864092...|    false| 11112.0|China is adding g...|         0.7655|
|2019-07-19 14:06|   6049.0|1.152187923908976...|    false|  1562.0|....State which I...|         0.8932|
|2019-07-19 14:16|  70874.0|1.152190440692756...|    false| 12980.0|....State which I...|         0.8932|
|2019-08-08 16:38|  85995.0|1.159473909827297...|    false| 17705.0|As your President...|         0.9273|
|2019-09-02 13:57|  15495.0|1.168493188056829...|    false|  4006.0|....Average hourl...|         0.7717|
|2019-09-16 13:47|  32931.0|1.173564172635914.

In [48]:
from pyspark.sql import functions as F

# Ensure both DataFrames have the `date` column in TIMESTAMP format
btc_price = btc_price.withColumn("date", F.to_timestamp("date", "yyyy-MM-dd HH:mm"))
filtered_tweets = filtered_tweets.withColumn("date", F.to_timestamp("date", "yyyy-MM-dd HH:mm"))

# Perform the LEFT JOIN on "date"
joined_df = btc_price.join(
    filtered_tweets,
    on="date",
    how="left"
)

# Show results
joined_df.show(10, truncate=False)



+-------------------+------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|date               |Timestamp   |Open   |High   |Low    |Close  |Volume     |favorites|id  |isRetweet|retweets|text|sentiment_score|
+-------------------+------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|2017-07-11 01:18:00|1.49972868E9|2419.94|2420.0 |2418.95|2418.95|4.07239513 |NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:19:00|1.49972874E9|2421.99|2422.0 |2419.18|2419.18|15.83349066|NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:20:00|1.4997288E9 |2419.91|2421.96|2419.18|2421.95|2.71926784 |NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:21:00|1.49972886E9|2421.96|2422.0 |2419.29|2422.0 |42.68201165|NULL     |NULL|NULL     |NULL    |NULL|NULL           |
|2017-07-11 01:22:00|1.49972892E9|2420.33|2422.98|2414.79|2414

In [49]:
# Define a 12-hour future time column
from pyspark.sql.functions import col, expr

# Create a self-join to find BTC price exactly 24 hours later how it changed over time
joined_df_with_future = joined_df.alias("current").join(
    joined_df.alias("future"),
    expr("future.date = current.date + INTERVAL 24 HOURS"),
    how="left"
).select(
    col("current.text"),
    col("current.date"),
    col("current.sentiment_score"),
    col("current.Close").alias("btc_current_price"),
    col("future.Close").alias("btc_price_24h_later")
)

# Show results
joined_df_with_future.show(10, truncate=False)


+----+-------------------+---------------+-----------------+-------------------+
|text|date               |sentiment_score|btc_current_price|btc_price_24h_later|
+----+-------------------+---------------+-----------------+-------------------+
|NULL|2012-01-01 11:01:00|NULL           |4.58             |5.0                |
|NULL|2012-01-01 11:03:00|NULL           |4.58             |5.0                |
|NULL|2012-01-01 11:08:00|NULL           |4.58             |5.0                |
|NULL|2012-01-01 11:11:00|NULL           |4.58             |5.0                |
|NULL|2017-07-11 01:21:00|NULL           |2422.0           |2326.26            |
|NULL|2017-07-11 01:22:00|NULL           |2414.79          |2324.96            |
|NULL|2017-07-11 01:24:00|NULL           |2409.43          |2325.0             |
|NULL|2017-07-11 01:28:00|NULL           |2411.43          |2329.67            |
|NULL|2022-01-11 07:04:00|NULL           |41687.21         |42743.46           |
|NULL|2022-01-11 07:06:00|NU

In [50]:
joined_df_with_future=joined_df_with_future.filter(F.col("sentiment_score").isNotNull())

In [51]:
joined_df_with_future.show()

+--------------------+-------------------+---------------+-----------------+-------------------+
|                text|               date|sentiment_score|btc_current_price|btc_price_24h_later|
+--------------------+-------------------+---------------+-----------------+-------------------+
|It is incredible ...|2018-12-17 14:27:00|         0.6893|           3234.5|            3495.05|
|....State which I...|2019-07-19 14:16:00|         0.8932|         10684.43|           10554.78|
|As your President...|2019-08-08 16:38:00|         0.9273|          11828.0|           11721.84|
|Producer prices i...|2019-09-16 13:47:00|        -0.7371|         10296.36|           10224.09|
|...The United Sta...|2019-09-16 13:47:00|         0.8908|         10296.36|           10224.09|
|Democrats are “he...|2020-09-16 16:30:00|        -0.6476|         10841.54|           10809.27|
|If I am sent a St...|2020-10-07 04:18:00|         0.6808|         10592.47|           10646.77|
|Nancy Pelosi coul...|2020-10-

In [33]:
joined_df_with_future.count()

29